In [1]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
from collections import namedtuple
from pyspark import SparkConf


def process_stream(record, spark):
    if not record.isEmpty():
        df = spark.createDataFrame(record) 
        df.show()
        
def aggregate_tags_count(new_values, total_sum):
    return sum(new_values) + (total_sum or 0)

def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
            globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
    return globals()['sqlContextSingletonInstance']


In [ ]:
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
       # Get spark sql singleton context from the current context
        sql_context = get_sql_context_instance(rdd.context)
        #print("Get spark sql singleton context from the current context ----------- %s -----------" % str(time))
        #sqlContext = SQLContext(sc)
        rows_rdd = rdd.map(lambda x: Row(tweet_id=x[0], tweet_screen_name=x[1], tweet_text=[2], source=x[3], device=x[4], tweet_country_code=x[5], tweet_createdat=x[6]))

        # create a DF from the Row RDD
        test_df = sql_context.createDataFrame(rows_rdd)
        test_df.show()
        # Register the dataframe as table
        test_df.registerTempTable("tweets")
        #getSQLContext = get_sql_context_instancet(rdd.context)
        
        
        # convert the RDD to Row RDD
        #row_rdd = rdd.map(lambda w: Row(word=w[0], word_count=w[1]))
        #map it
        #rows_rdd = rdd.map(lambda x: Row(tweet_id=x[0], tweet_screen_name=x[1], tweet_text=[2], source=x[3], device=x[4], tweet_country_code=x[5], tweet_createdat=x[6]))
        
        
        #rows_rdd = rdd.map(lambda x: Row(tweet_id=x[0], tweet_screen_name=x[1], tweet_text=[2], source=x[3], device=x[4], tweet_country_code=x[5], tweet_createdat=x[6]))
        
        #rows_rdd.foreachRDD(lambda rdd: process_stream(rdd, spark))

   

   
        # get the top 10 test from the table using SQL and print them
        # get data into a dataframe
        
        testContextDF = sql_context.sql("select tweet_id , tweet_screen_name, tweet_text, source, device, tweet_country_code, tweet_createdat from tweets order by tweet_id desc limit 200")
        testContextDF.coalesce(1).write.format(('com.databricks.spark.csv').mode('overwrite').option("header", "true").csv("testData.csv") )

        #return testContext
        #new_testDF = testContext.toPandas()
        #test_df.head()
        #hashtag_counts_df.coalesce(1).write.format('com.databricks.spark.csv').mode('overwrite').option("header", "true").csv("/Users/girishdurgaiah/hashtag_file.csv") 
   
        #country_counts_df = sql_context.sql("select word as country_code, word_count as tweet_count from test where word like 'CC%'order by word_count desc limit 10")
        #country_counts_df.show()
        #country_counts_df.coalesce(1).write.format('com.databricks.spark.csv').mode('overwrite').option("header", "true").csv("/Users/girishdurgaiah/country_file.csv")
   
        #device_df = sql_context.sql("select word as device, word_count as device_count from test where word like 'TS%'order by word_count desc limit 10")
        #device_df.show()
        #device_df.coalesce(1).write.format('com.databricks.spark.csv').mode('overwrite').option("header", "true").csv("/Users/girishdurgaiah/device_file.csv")
           
    except:
        pass



# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")

# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)
sqlContext = SQLContext(sc)


# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")

# read data from port 5555
dataStream = ssc.socketTextStream("localhost",5550)
dataStream.foreachRDD(process_rdd)

#lines = dataStream.flatMap(lambda text: text.split(" "))
#lines = dataStream.window(20)
#fields = ("tweet_id" , "tweet_screen_name", "tweet_text", "source", "device", "tweet_country_code", "tweet_createdat")
#(lines.flatMap(lambda rec: Tweet( rec[0], rec[1], rec[2], rec[3],rec[4], rec[5], rec[6])).map(lambda rec: Tweet(rec[0], rec[1], rec[2], rec[3], rec[4], rec[5], rec[6])).foreachRDD(lambda rdd: rdd.toDF().limit(200).registerTempTable("tweets")))


# split each tweet into words
#words = dataStream.flatMap(lambda line: line.split(" "))
              
# filter the words to get only test, then map each hashtag to be a pair of (hashtag,1)
#test = words.map(lambda x: (x, 1)) 

# adding the count of each hashtag to its last count
#tags_totals = test.updateStateByKey(aggregate_tags_count)

# do processing for each RDD generated in each interval
#tags_totals.foreachRDD(process_rdd)
#lines.foreachRDD(lambda rdd: rdd.toDF())
#lines.map( lambda rec: Tweet( rec[0], rec[1], rec[2], rec[3],rec[4], rec[5], rec[6]))#.foreachRDD(lambda rdd: rdd.toDF())


# start the streaming computation
ssc.start()

# wait for the streaming to finish
#ssc.awaitTermination()

#end
#scc.stop()

In [ ]:
#ssc.stop()

In [ ]:
top200_tweets = sqlContext.sql('select tweet_id, tweet_screen_name, tweet_text, source, device, tweet_country_code, tweet_createdat from tweets order by tweet_id limit 200')

In [ ]:
testContextDF.head()

In [ ]:
# wait for the streaming to finish
ssc.awaitTermination()

In [ ]:
testContextDF = sqlContext.sql("select tweet_id , tweet_screen_name, tweet_text, source, device, tweet_country_code, tweet_createdat from tweets order by tweet_id desc limit 200")


In [2]:
# define a function to compute sentiments of the received tweets
def get_prediction(tweet_text):
    try:
    # filter the tweets whose length is greater than 0
        rdd = tweet_text.filter(lambda x: len(x) > 0)
    # create a dataframe with column name 'tweet' and each row will contain the tweet
        rows_rdd = rdd.map(lambda x: Row(tweet_id=x[0], tweet_screen_name=x[1], tweet_text=[2], source=x[3], device=x[4], tweet_country_code=x[5], tweet_createdat=x[6]))
    # create a spark dataframe
        wordsDataFrame = spark.createDataFrame(rows_rdd)
    # create a table
        wordsDataFrame.registerTable("tweets_table")
        testDF = SQLContext.sql("select tweet_id, tweet_screen_name, tweet_text, source, device, tweet_country_code, tweet_createdat from tweets_table order by tweet_id limit 200")
    
    except : 
        print('No data')

In [3]:
# create spark configuration
conf = SparkConf()
conf.setAppName("checkpoint_TwitterApp")

# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
#Streamin contest
ssc = StreamingContext(sc, batchDuration=3)
lines = ssc.socketTextStream("localhost", 5551)
words = lines.flatMap(lambda x: x.split(""))
words.foreachRDD(get_prediction)
ssc.start()
ssc.awaitTermination()

No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data
No data


KeyboardInterrupt: 